In [7]:
import cv2
from matplotlib import pyplot as plt
import albumentations as a
import os
import json
import copy
import numpy as np
#import shutil

In [2]:
def get_dataset_dict(imgDir, JsonPath):
    """Get dataset dictionary for an image.
    @param imgDir: The source image.
    @JsonPath size: Get original Json Path.
    @return The dataset dictionary.
    """
    # Get list of Imgs
    images_file = os.listdir(imgDir)

    # Get JSON File
    with open(JsonPath) as f:
        Annotation_Json = json.load(f) 

    dataset_dict=[]
    for TargetImg in images_file:

        Target_ID = None
        for img in Annotation_Json['images']:
            if img['file_name'] == TargetImg:
                Target_ID = img['id']
                break

        img_dict = {
            "file_name": TargetImg,
            "annotations": [{"segmentation": [], "bbox" : [],"category_id": 0}]
        }
        for img in Annotation_Json["annotations"]:
            if img["image_id"] == Target_ID:
                img_dict["annotations"][0]["segmentation"].append(img["segmentation"][0])
                img_dict["annotations"][0]["bbox"].append(img["bbox"])
                #img_dict["annotations"][0]["category_id"].append(0)

        img_dict["annotations"][0]["segmentation"] = list(np.concatenate(img_dict["annotations"][0]["segmentation"]).flat)
        img_dict["annotations"][0]["bbox"] = img_dict["annotations"][0]["bbox"]

        dataset_dict.append(img_dict)
    
    return dataset_dict

In [3]:
def Aug_IMGs(imgDir, JsonPath, tfs, out_Img_P, out_Json_P):
    ''' Get Augmented images with annotations.
    @param imgDir: The source image.
    @JsonPath size: Get original Json Path.
    @tfs: List of transformations,
    @out_Img_P: Output of augmented images
    @out_Json_P: Output of json augmented images
    @return images with annotations.
    
    '''  
    # Get list of Imgs
    images_file = os.listdir(imgDir)

    # Get JSON File
    with open(JsonPath) as f:
        Annotation_Json = json.load(f) 
        Annotation_Output= copy.deepcopy(Annotation_Json)

    # Loop over all the images in the folder
    for TargetImg in images_file:
        
        # Read img
        img_in_RGB = cv2.imread(imgDir+f"/{TargetImg}")
        #img_in_RGB = cv2.cvtColor(img_in_RGB, cv2.COLOR_BGR2RGB)

        # Find the corresponding IDs in the json file
        Target_ID = None
        for img in Annotation_Json['images']:
            if img['file_name'] == TargetImg:
                Target_ID = img['id']
                break
        
        assert Target_ID != None, f"missing annotations in json file for {TargetImg}"

        # Find all the annotations for the image 
        Image_categories = []
        Image_bboxes = []
        Image_keypoints  = []
        keypoints_separator = []
        for img in Annotation_Json["annotations"]:
            if img["image_id"] == Target_ID:
                # transform the list of points in a list of tuples
                # ie change the keypoints format for 'xy'
                annot = img["segmentation"][0]
                keypoints = [(annot[i], annot[i+1]) for i in range(0, len(annot), 2)]
                Image_keypoints.extend(keypoints)
                keypoints_separator.append(len(keypoints))
                # Same for bboxes
                bbox = img["bbox"]
                Image_bboxes.append(bbox)
                # Get object category
                Image_categories.append(img["category_id"])

        # Apply the transformation
        for t in range(len(tfs)):
            tf = tfs[t]

            output = tf(
                image = img_in_RGB, 
                bboxes = Image_bboxes, 
                category_ids = Image_categories, 
                keypoints = Image_keypoints
                )

            new_image = output["image"]
            new_bbox = output['bboxes']
            new_keypoints = output['keypoints']

            # Convert back keypoints
            new_keypoints = list(np.concatenate([[i[0], i[1]] for i in new_keypoints]).flat)

            # Create a new dictionary 'Img_Dict' for the augmented image
            Img_Dic = {
                "id" : len(Annotation_Output["images"]) + 1,
                "width": new_image.shape[0],
                "height": new_image.shape[0],
                "file_name": f"Aug{t+1}_{TargetImg}"
            }

            # Append this dictionary to the output json
            Annotation_Output["images"].append(Img_Dic)

            # Create a new dictionary 'Annot_Dict' for the augmented image
            for id_kp in range(len(keypoints_separator)):
                keypoints = new_keypoints[0:2*keypoints_separator[id_kp]]
                new_keypoints = new_keypoints[2*keypoints_separator[id_kp]:]
                Annot_Dict = {
                    "id": len(Annotation_Output["annotations"]) + 1,
                    "image_id": Img_Dic["id"],
                    "category_id": 1,
                    "segmentation": [[round(kpts,2) for kpts in keypoints]],
                    "area": None,
                    "bbox": [round(bb,2) for bb in new_bbox[id_kp]],
                    "is_crowd": 0
                }

                Annotation_Output["annotations"].append(Annot_Dict)

            # Save the image
            #shutil.copyfile(os.path.join(imgDir, TargetImg), os.path.join(out_Img_P, TargetImg))
            cv2.imwrite(os.path.join(out_Img_P, Img_Dic["file_name"]) , new_image)

    # Save json
    with open(out_Json_P, 'w') as f:
        json.dump(Annotation_Output, f, indent = 4)

In [8]:
transform1 = a.Compose([
    a.RandomCrop(width=256, height=256),
    a.HorizontalFlip(p=1),
    a.RandomBrightnessContrast(p=1)], 
    bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
    keypoint_params=a.KeypointParams(format='xy'))

# Apply Contrast Limited Adaptive Histogram Equalization to the input image.
transform2 = a.Compose([
    a.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=1),
    a.ChannelDropout(channel_drop_range=(1, 1), fill_value=0, p=1)], 
    bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
    keypoint_params=a.KeypointParams(format='xy'))

# Blur the input image using a random-sized kernel.
transform3 = a.Compose([
    a.Blur(blur_limit=2, p=1),
    a.ChannelShuffle(p=1)], 
    bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
    keypoint_params=a.KeypointParams(format='xy'))

# Blur the input image using a Gaussian filter with a random kernel size.
transform4 = a.Compose([
    a.GaussianBlur(blur_limit=(3, 3), sigma_limit=0, p=1),
    a.ColorJitter(brightness=0.2, contrast=0.4, saturation=0.3, hue=0.5, p=1)], 
    bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
    keypoint_params=a.KeypointParams(format='xy'))

# Blur the input image using a median filter with a random aperture linear size.
transform5 = a.Compose([
    a.MedianBlur(blur_limit=3, p=1),
    a.FancyPCA(alpha=0.1, p=1)], 
    bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
    keypoint_params=a.KeypointParams(format='xy'))

# Apply motion blur to the input image using a random-sized kernel.
transform6 = a.Compose([
    a.MotionBlur(blur_limit=3, p=1),
    a.ToSepia(p=1)], 
    bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
    keypoint_params=a.KeypointParams(format='xy'))

#Apply glass noise to the input image.
transform7 = a.Compose([
    a.GlassBlur(sigma=0.7, max_delta=4, iterations=2, mode='fast', p=1),
    a.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1)], 
    bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']), 
    keypoint_params=a.KeypointParams(format='xy'))

# Applys resize, , vertical flip and noise
transform8 = a.Compose([
    a.Resize(width=320, height=320),
    a.VerticalFlip(p=1),
    a.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1)], 
    bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
    keypoint_params=a.KeypointParams(format='xy'))

# Applys resize, random rotate and gaussian noise
transform9 = a.Compose([
    a.Resize(width=352, height=352, interpolation=cv2.INTER_LINEAR, p=1),
    a.RandomRotate90(p=0.85),
    a.GaussNoise(var_limit=(10.0, 50.0), mean=0, p=1)], 
    bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
    keypoint_params=a.KeypointParams(format='xy'))

# Applys equalize, Hue saturation and ISO noisee
transform10 = a.Compose([
    a.Equalize(mode='cv', by_channels=True, mask=None, mask_params=(), p=1),
    a.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1),
    a.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1)], 
    bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
    keypoint_params=a.KeypointParams(format='xy'))
    

# Applys horizontal and vertical flip, random brightness contrast
transform11 = a.Compose([
                a.HorizontalFlip(p=0.5),
                a.VerticalFlip(p=0.5),
                a.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1)
                #A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), always_apply=True),
                ],
                bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
                keypoint_params=a.KeypointParams(format='xy')
                )    

# Applys Random Rain and blur
transform12 = a.Compose([
                a.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(200, 200, 200), 
                blur_value=7, brightness_coefficient=0.7, rain_type=None, always_apply=False, p=0.8)],
                bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
                keypoint_params=a.KeypointParams(format='xy')
                ) 
# Applys Random Snow
transform13 = a.Compose([
                a.RandomSnow (snow_point_lower=0.2, snow_point_upper=0.5, brightness_coeff=2.5, always_apply=False, p=0.5)],
                bbox_params=a.BboxParams(format='coco', label_fields=['category_ids']),
                keypoint_params=a.KeypointParams(format='xy')
                )    

In [9]:
tfs = [transform4, transform5, transform6, transform7,transform8, transform9,transform10, transform11,transform12, transform13]

In [10]:
in_Path ="./data/annotations/TahitiFaaa_2013_Squarred/"
in_Ann_Path ="./data/annotations/annotation_2013_Faa.json" 
Out_Path = "./data/annotations//Augmentation/Out_img3/"
out_Annotation_p = "./data/annotations/output_9.json" 

Classes_Name = ["is-coral"]


Aug_IMGs(in_Path, in_Ann_Path, tfs, Out_Path, out_Annotation_p)